In [1]:
import pandas as pd

In [2]:

deseq_data = pd.read_csv(r"DEseq2.csv")
deseq_2_data = pd.read_csv("FinalGeneList_1.5_fold_NoTrkA.txt", names = ["ensembl_gene_id_version", "external_gene_name"], sep="\t") #csv has no header
edge_data = pd.read_csv(r"EdgeR.csv")
limma_data = pd.read_csv(r"limma-voom.csv")

In [3]:
deseq_data.head()

,Unnamed: 0,external_gene_name,log2FoldChange,padj,gene_biotype,description,ensembl_gene_id_version
0,1,AL139011.2,6.568743,0.040201,protein_coding,novel protein,ENSG00000258465.8
1,2,PLXDC1,3.570140,0.032559,protein_coding,plexin domain containing 1 [Source:HGNC Symbol...,ENSG00000161381.14
2,3,DIPK2A,2.576218,0.036660,protein_coding,divergent protein kinase domain 2A [Source:HGN...,ENSG00000181744.9
3,4,VWA7,2.350473,0.040201,protein_coding,von Willebrand factor A domain containing 7 [S...,ENSG00000204396.11
4,5,ITGA11,2.151866,0.016861,protein_coding,integrin subunit alpha 11 [Source:HGNC Symbol;...,ENSG00000137809.17


In [4]:
deseq_2_data.head()

,ensembl_gene_id_version,external_gene_name
0,ENSG00000198400,NTRK1
1,ENSG00000198825,INPP5F
2,ENSG00000249839,AC011330.1
3,ENSG00000249896,LINC02495
4,ENSG00000180787,ZFP3


In [5]:
edge_data.head()

,Unnamed: 0,external_gene_name,log2FoldChange,padj,gene_biotype,description,ensembl_gene_id_version
0,1,FAM220A,6.756461,4.000000e-28,protein_coding,family with sequence similarity 220 member A [...,ENSG00000178397.13
1,2,AL159163.1,5.078716,9.976791e-03,protein_coding,novel protein,ENSG00000249141.1
2,3,RNU6-33P,4.929061,1.612806e-02,snRNA,"RNA, U6 small nuclear 33, pseudogene [Source:H...",ENSG00000207524.1
3,4,HAPLN4,4.788466,2.624322e-02,protein_coding,hyaluronan and proteoglycan link protein 4 [So...,ENSG00000187664.9
4,5,ARL2-SNX15,2.724237,3.728628e-03,protein_coding,ARL2-SNX15 readthrough (NMD candidate) [Source...,ENSG00000273003.1


In [6]:
limma_data.head()

,Unnamed: 0,external_gene_name,log2FoldChange,padj,gene_biotype,description,ensembl_gene_id_version
0,1,VWA7,2.431279,0.047555,protein_coding,von Willebrand factor A domain containing 7 [S...,ENSG00000204396.11
1,2,ITGA11,2.047691,0.042485,protein_coding,integrin subunit alpha 11 [Source:HGNC Symbol;...,ENSG00000137809.17
2,3,IGSF10,1.717703,0.040909,protein_coding,immunoglobulin superfamily member 10 [Source:H...,ENSG00000152580.8
3,4,CHODL,1.706261,0.013145,protein_coding,chondrolectin [Source:HGNC Symbol;Acc:HGNC:17807],ENSG00000154645.14
4,5,SH2D3C,1.174026,0.048801,protein_coding,SH2 domain containing 3C [Source:HGNC Symbol;A...,ENSG00000095370.20


In [7]:
deseq_data_genes = pd.Series(deseq_data.external_gene_name)
deseq_2_data_genes = pd.Series(deseq_2_data.external_gene_name)
edge_data_genes = pd.Series(edge_data.external_gene_name)
limma_data_genes = pd.Series(limma_data.external_gene_name)

In [8]:
###### expected 36, 68, 85, 26 respectively
gene_sets = [deseq_data_genes, deseq_2_data_genes, edge_data_genes, limma_data_genes]
for i in gene_sets:
    print(len(i))

36
69
85
26


The deseq_2 gene list contains 69 genes. 
According to section 5.2.2 of Arthurs final report it shoul be 68 genes.
The first gene is NTRK1 (TrkA), remove

In [9]:
deseq_2_data_genes = deseq_2_data_genes.drop(0)  # remove NTRK1 

In [10]:
gene_list = deseq_data_genes.append(deseq_2_data_genes, ignore_index = True).append(edge_data_genes, ignore_index = True).append(limma_data_genes, ignore_index = True)
print(len(gene_list)) # correct length

215


In [11]:
gene_list = gene_list.drop_duplicates(keep="first")
print(gene_list) #will be 169 when adding the four CoV genes

0      AL139011.2
1          PLXDC1
2          DIPK2A
3            VWA7
4          ITGA11
          ...    
191        IGSF10
192         CHODL
207         PLPP2
209        CLDN23
213         KCNQ3
Name: external_gene_name, Length: 163, dtype: object


In [12]:
gene_list.is_unique  #quality control to check duplicates are removed

True

In [33]:
#gene_list_to_file = gene_list.append(pd.Series(["ACE2", "CD147", "PPIA", "PPIB"])) #append the 4 CoV2 genes
#gene_list_to_file.to_csv("deg_gene_name.csv", index = False)

The Below cell is the same but instead of by merging by gene_name does so by ensemble_gene_id_version:

In [14]:
#The same but by ensembl_gene_id_version
deseq_data_id = pd.Series(deseq_data.ensembl_gene_id_version)
deseq_2_data_id = pd.Series(deseq_2_data.ensembl_gene_id_version)
edge_data_id = pd.Series(edge_data.ensembl_gene_id_version)
limma_data_id = pd.Series(limma_data.ensembl_gene_id_version)

deseq_2_data_id = deseq_2_data_id.drop(0)

id_list = deseq_data_id.append(deseq_2_data_id, ignore_index = True).append(edge_data_id, ignore_index = True).append(limma_data_id, ignore_index = True)
print(len(id_list)) # correct length
id_list = id_list.drop_duplicates(keep="first")
print(id_list) #will be 17



215
0       ENSG00000258465.8
1      ENSG00000161381.14
2       ENSG00000181744.9
3      ENSG00000204396.11
4      ENSG00000137809.17
              ...        
192    ENSG00000154645.14
193    ENSG00000095370.20
207    ENSG00000141934.10
209     ENSG00000253958.2
213    ENSG00000184156.17
Name: ensembl_gene_id_version, Length: 176, dtype: object


Hence, some gene names map to more than one ENS code
176 (compared to 163)


Below: to find which genes map to multiple ENS codes - 

In [16]:
#create a dic of the 163 gene names with [] values
name_to_ens = dict()
for i in list(gene_list):
    name_to_ens.setdefault(i, [])

In [17]:
#fx to add unique ensembl id's that map to gene names

def get_ens(data):
    for index, name in enumerate(data.external_gene_name):
        for j in name_to_ens:
            if name == j:
                if data.ensembl_gene_id_version[index] not in name_to_ens[j]:
                    name_to_ens[j].append(data.ensembl_gene_id_version[index])
                   

In [18]:
for i in [deseq_data, deseq_2_data, edge_data, limma_data]:
    get_ens(i)

In [19]:
#This cell just checks the above code works (should be 176)
count= 0
for i in name_to_ens:
    count += len(name_to_ens[i])
print(count)

176


In [20]:
for i in name_to_ens:
    if len(name_to_ens[i]) > 1:
        print(i, name_to_ens[i], "\n")

FNBP1 ['ENSG00000187239.17', 'ENSG00000187239'] 

ZNF423 ['ENSG00000102935.11', 'ENSG00000102935'] 

SIX3 ['ENSG00000138083.5', 'ENSG00000138083'] 

PCDHB14 ['ENSG00000120327.6', 'ENSG00000120327'] 

TNNT1 ['ENSG00000105048.17', 'ENSG00000105048'] 

BCAN ['ENSG00000132692.19', 'ENSG00000132692'] 

DLG2 ['ENSG00000150672.17', 'ENSG00000150672'] 

TRIM5 ['ENSG00000132256.19', 'ENSG00000132256'] 

DENND1C ['ENSG00000205744.10', 'ENSG00000205744'] 

PCDHB2 ['ENSG00000112852.7', 'ENSG00000112852'] 

RGS10 ['ENSG00000148908', 'ENSG00000148908.15'] 

SH2D3C ['ENSG00000095370', 'ENSG00000095370.20'] 

FRAS1 ['ENSG00000138759', 'ENSG00000138759.19'] 



There are 13 gene names with two codes (163 +13 = 176). These ID's need to further investigated